In [1]:
import torch

In [ ]:
import nltk
import pytesseract
import cv2
import streamlit as st
import os
import tempfile
import spacy
from pathlib import Path
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import datefinder
from datetime import datetime
from keybert import KeyBERT
from PyPDF2 import PdfReader
import docx2txt


In [3]:
from transformers import pipeline

In [64]:
print(torch.cuda.is_available())  # Should print: True


True


In [5]:
print("Device name:", torch.cuda.get_device_name(0))

Device name: NVIDIA GeForce RTX 3050 Laptop GPU


In [6]:
#python -m nltk.downloader punkt stopwords
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to C:\Users\Harshal
[nltk_data]     Sharma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Harshal
[nltk_data]     Sharma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Harshal
[nltk_data]     Sharma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Harshal
[nltk_data]     Sharma\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Harshal
[nltk_data]     Sharma\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
!pip install pdfminer.six python-docx pytesseract opencv-python keybert sentence-transformers streamlit --progress-bar on --no-cache-dir -v


Using pip 25.1 from C:\Anaconda\envs\meta-gen\lib\site-packages\pip (python 3.10)


In [8]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"


In [9]:
stop_words = set(stopwords.words('english'))

In [82]:
base_dir = 'data'

In [11]:
%pwd

'c:\\Users\\Harshal Sharma\\Desktop\\meta-gen\\research'

In [12]:
cd ..

c:\Users\Harshal Sharma\Desktop\meta-gen


In [83]:
files = os.listdir(base_dir)

In [84]:
files

['THE-INTELLIGENT-INVESTOR.pdf', 'uploads']

In [85]:
path = os.path.join(base_dir,files[1])

In [86]:
path

'data\\uploads'

In [87]:
file = os.listdir(path)

In [88]:
file

['MARS OPEN PROJECTS 2025 (1).pdf']

In [89]:
path = os.path.join(path,file[0])

In [90]:
path

'data\\uploads\\MARS OPEN PROJECTS 2025 (1).pdf'

In [91]:

def extract_text(path):
    file_ext = Path(path).suffix.lower()

    if file_ext == '.pdf':
        reader = PdfReader(path)
        return "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])

    elif file_ext == '.docx':
        return docx2txt.process(path)

    elif file_ext == '.txt':
        with open(path, 'r', encoding='utf-8') as file:
            return file.read()

    elif file_ext in ['.jpg', '.jpeg', '.png']:
        img = cv2.imread(path)
        return pytesseract.image_to_string(img)

    return ""


In [92]:
def clean_and_lemmatize_nltk(text):
    # Remove special characters and normalize spaces
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip().lower()

    # Tokenize
    words = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in words]

    return " ".join(lemmatized)


In [93]:
text = extract_text(path)
cleaned_text = clean_and_lemmatize_nltk(text)

In [94]:
cleaned_text

'open project 2025 aiml problem statement 1 design implement robust pipeline perform emotion classification speech data objective objective project design implement end toend pipeline emotion classification using speech data system leverage audio processing technique machine learningdeep learning model accurately identify categorize emotional state conveyed speechsong dataset dataset description file available n given link note download file filename downloaded informed doubt group audio dataset evaluation criterion model accuracy j udged validation data condition confusion matrix judging criterion f1 score greater 80 accuracy class greater 75 overall accuracy greater 80 apart validation data checking accuracy custom test dataset shared publicly deliverable 1 github repository containing ipynb notebook full running code b trained model c python script file model tested feeding test data inclu de clear detailed readmemd including project description pre processing methodology model pipe

In [95]:
def extract_title(text):
    # Split by lines
    lines = text.strip().split("\n")

    # Clean lines and remove empty ones
    lines = [line.strip() for line in lines if line.strip()]

    if not lines:
        return "Unknown Title"

    # Heuristic: longest first line with reasonable length (5–100 chars)
    for line in lines:
        if 5 < len(line) < 100 and line[0].isupper():
            return line

    return lines[0]  # fallback


In [96]:
def extract_author(text):
    # Step 1: First try to detect lines like "By Benjamin Graham"
    lines = text.strip().split('\n')
    for line in lines:
        line = line.strip()
        match = re.search(r"\b(By|Written by|Author:)\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)+)", line)
        if match:
            return match.group(2)

    # Step 2: Fallback to NER (find first PERSON entity)
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            return ent.text

    return "Unknown Author"

In [97]:
def extract_date(text):
    matches = list(datefinder.find_dates(text))
    
    # Optional: filter future dates if not useful
    matches = [d for d in matches if d.year <= datetime.now().year]

    if matches:
        # Return the first detected date (assumed as publishing/mention)
        return matches[0].strftime("%Y-%m-%d")
    
    return "Unknown Date"

In [98]:
kw_model = KeyBERT(model='all-MiniLM-L6-v2')  # Fast & good quality

def extract_keywords(text, top_n=5):
    keywords = kw_model.extract_keywords(text, top_n=top_n, stop_words='english')
    # keywords is a list of tuples: (keyword, score)
    return [kw[0] for kw in keywords]

In [99]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0)

Device set to use cuda:0


In [100]:
def split_text_into_chunks(text, max_chunk_size=3000):
    # Clean up whitespace
    text = text.replace('\n', ' ').strip()

    chunks = []
    while len(text) > max_chunk_size:
        # Find nearest period before max_chunk_size
        split_index = text.rfind('.', 0, max_chunk_size)
        if split_index == -1:
            split_index = max_chunk_size  # fallback
        chunks.append(text[:split_index + 1].strip())
        text = text[split_index + 1:].strip()

    if text:
        chunks.append(text)

    return chunks


In [101]:
def generate_summary(text, min_len=40, max_len=150):
    chunks = split_text_into_chunks(text)
    print(f"Total Chunks: {len(chunks)}")

    final_summary = ""

    for i, chunk in enumerate(chunks):
        summary = summarizer(chunk, max_length=max_len, min_length=min_len, do_sample=False)[0]['summary_text']
        final_summary += summary.strip() + " "

    return final_summary.strip()


In [102]:
summary_output = generate_summary(cleaned_text)
print("Final Summary:\n")
print(summary_output)

Total Chunks: 3


Your max_length is set to 150, but your input_length is only 18. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)


Final Summary:

open project 2025 aiml problem statement 1 design implement robust pipeline perform emotion classification speech data objective objective project design implement end toend pipeline emotion classification using speech data system. 2 fully functional web app hosted using stream lit receive audio file input return classified emotion group link 2 automated meta data generation objective project aim develop automated metadata generation system system enhance document discoverability classification subsequent analysis. Project is a web application that allows users to upload files. The project includes a demo video demonstrating core feature working live. The video was uploaded to YouTube and is part of a larger project. The code is available on GitHub. your team feel free to use open source tools. libraries ensure implementation logic. al done you. Use these tools to help your team get the most out of open source software. Use this tool to help you get the best out of your

In [103]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)  # use -1 for CPU

Device set to use cuda:0


In [104]:
def detect_category_from_chunks(text, candidate_labels=None, max_chunk_size=1000):
    from collections import Counter

    if candidate_labels is None:
        candidate_labels = ["Finance", "Health", "Education", "Politics", "Technology", "History", "Philosophy", "Biography", "Science", "Fiction"]

    # Chunk the text manually
    chunks = [text[i:i+max_chunk_size] for i in range(0, len(text), max_chunk_size)]
    
    scores = Counter()

    for chunk in chunks:
        result = classifier(chunk, candidate_labels)
        top_label = result['labels'][0]
        confidence = result['scores'][0]
        scores[top_label] += confidence

    # Pick the category with highest total confidence
    best_category = scores.most_common(1)[0][0]
    return best_category, dict(scores)


In [ ]:
text = extract_text(path)                   # Raw full text from uploaded file
title = extract_title(text)                # On raw
author = extract_author(text)              # On raw
date = extract_date(text)                  # On raw
summary = generate_summary(text)           # On raw
category, scores = detect_category_from_chunks(text)  # On raw
keywords = extract_keywords(text)          # Can use raw or cleaned

# Optional
cleaned_text = clean_and_lemmatize_nltk(text)  # Only for extra features


In [106]:
def generate_metadata(text):
    metadata = {}

    # Step-by-step extraction
    metadata["title"] = extract_title(text)
    metadata["author"] = extract_author(text)
    metadata["date"] = extract_date(text)
    metadata["summary"] = generate_summary(text)
    metadata["category"], metadata["category_scores"] = detect_category_from_chunks(text)
    metadata["keywords"] = extract_keywords(text)

    return metadata


In [107]:
raw_text = extract_text(path)
metadata = generate_metadata(raw_text)

from pprint import pprint
pprint(metadata)


Total Chunks: 3
{'author': 'Deliverables',
 'category': 'Technology',
 'category_scores': {'Health': 0.28131720423698425,
                     'Technology': 3.31836499273777},
 'date': '2025-06-22',
 'keywords': ['voice', 'classification', 'speech', 'audio', 'emotion'],
 'summary': 'The project aims to design and implement an end -to-end pipeline '
            'for emotion classification using speech data. The system will '
            'leverage audio processing techniques and machine learning/deep '
            'learning models to accurately identify and categorize emotional '
            'states conveyed in speech/song. The project also aims to develop '
            'an automated metadata generation system. Design and develop a '
            'real-time collaborative whiteboard application that allows  '
            'multiple users to draw, write, and interact simultaneously, '
            'replicating the  experience of a physical whiteboard on the web. '
            'Frontend: React